# **Bloco 1: Importação de Bibliotecas e Carregamento dos Dados**

In [ ]:
# Instalação de pacotes necessários
%pip install pandas scikit-learn nltk openpyxl

In [7]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from openpyxl import Workbook
from openpyxl.styles import PatternFill
import logging

# Configurar logging para acompanhar o processo
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Baixar stopwords do NLTK (necessário na primeira execução)
try:
    nltk.download('stopwords', quiet=True)
    logging.info("Stopwords do NLTK baixadas com sucesso")
except Exception as e:
    logging.warning(f"Erro ao baixar stopwords: {e}")

# Caminho do arquivo com formato padronizado
file_path = r'C:\\Users\\FUDO\\OneDrive - PETROBRAS\\Documentos\\Python\\Problema do Leandro\\ContratoServico-FormulasR 1.xlsx'

# Verificar se o arquivo existe
if not os.path.exists(file_path):
    logging.error(f"Arquivo não encontrado: {file_path}")
    raise FileNotFoundError(f"O arquivo {file_path} não existe")

# Carregamento dos dados com tratamento de erro
try:
    df = pd.read_excel(file_path)
    logging.info(f"Arquivo carregado com sucesso. Dimensões: {df.shape}")
except Exception as e:
    logging.error(f"Erro ao carregar o arquivo: {e}")
    raise

# Exibir informações básicas sobre o dataframe
print(f"Total de linhas: {df.shape[0]}")
print(f"Total de colunas: {df.shape[1]}")
print(f"Colunas no DataFrame: {', '.join(df.columns)}")

2025-04-11 13:33:16,225 - INFO - Stopwords do NLTK baixadas com sucesso
2025-04-11 13:33:17,325 - INFO - Arquivo carregado com sucesso. Dimensões: (2571, 55)


Total de linhas: 2571
Total de colunas: 55
Colunas no DataFrame: Classificação, Sub-Classificação, Projeto, Contrato Juridico, Contrato SAP, Item, Linha, Descrição Linha de Serviço, Unidade de Medida, Qtde Prevista, Qtde Realizada, Preço Base, Valor Líquido Realizado, Valor PIS, Valor COFINS, Valor ICMS, Valor Bruto Realizado, Fornecedor, Data Base, Moeda, idLinhaServico, Indice1, Peso1, FatorK1, Indice2, Peso2, FatorK2, Indice3, FatorK3, Peso3, Indice4, Peso4, FatorK4, Indice5, Peso5, FatorK5, Indice6, Peso6, FatorK6, Análise.idLinhaServico, ACG, EQN, IGP, INS, IPAMB, IPAMQ, IPAOG, IPAPQ, IPCA, IPI, IPM, MME, MOE, TPO, Total


# **Bloco 2: Análise Exploratória e Pré-processamento das Features**

In [8]:
# Verificar valores nulos nas colunas relevantes
print("\nQuantidade de valores nulos por coluna:")
print(df.isnull().sum())

# Selecionar colunas relevantes
features = ['Descrição Linha de Serviço', 'Classificação', 'Fornecedor', 'Moeda', 'Item']
targets = ['Indice1', 'Peso1', 'FatorK1', 'Indice2', 'Peso2', 'FatorK2', 
           'Indice3', 'Peso3', 'FatorK3', 'Indice4', 'Peso4', 'FatorK4', 
           'Indice5', 'Peso5', 'FatorK5', 'Indice6', 'Peso6', 'FatorK6']

# Verificar se todas as colunas existem no DataFrame
missing_cols = [col for col in features + targets if col not in df.columns]
if missing_cols:
    logging.warning(f"Colunas não encontradas no DataFrame: {missing_cols}")

# Tratar valores nulos nas features de texto e categóricas
for col in features:
    if col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].fillna('')
            logging.info(f"Valores nulos na coluna '{col}' preenchidos com ''")
        else:
            df[col] = df[col].fillna(0)
            logging.info(f"Valores nulos na coluna '{col}' preenchidos com 0")

# Pré-processamento de texto para 'Descrição Linha de Serviço'
try:
    stop_words = set(stopwords.words('portuguese'))
    tfidf = TfidfVectorizer(stop_words=list(stop_words), max_features=50)
    
    # Garantir que não haja valores nulos antes do TF-IDF
    if 'Descrição Linha de Serviço' in df.columns:
        desc_tfidf = tfidf.fit_transform(df['Descrição Linha de Serviço']).toarray()
        desc_df = pd.DataFrame(desc_tfidf, columns=[f"tfidf_{i}" for i in range(desc_tfidf.shape[1])])
        logging.info(f"TF-IDF aplicado com sucesso. Dimensões: {desc_df.shape}")
    else:
        logging.error("Coluna 'Descrição Linha de Serviço' não encontrada no DataFrame")
        desc_df = pd.DataFrame()
except Exception as e:
    logging.error(f"Erro ao processar TF-IDF: {e}")
    desc_df = pd.DataFrame()

# Codificação One-Hot para variáveis categóricas com tratamento de erro
try:
    cat_cols = [col for col in ['Classificação', 'Fornecedor', 'Moeda'] if col in df.columns]
    if cat_cols:
        # Remover valores NaN ou vazios antes da codificação
        df_cat = df[cat_cols].fillna('MISSING')
        
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        cat_encoded = encoder.fit_transform(df_cat)
        cat_df = pd.DataFrame(cat_encoded, 
                            columns=encoder.get_feature_names_out(cat_cols))
        logging.info(f"Codificação One-Hot aplicada com sucesso. Dimensões: {cat_df.shape}")
    else:
        logging.warning("Nenhuma coluna categórica encontrada para codificação")
        cat_df = pd.DataFrame()
except Exception as e:
    logging.error(f"Erro na codificação One-Hot: {e}")
    cat_df = pd.DataFrame(index=range(len(df)))

# Tratar 'Item' como numérico com verificação
try:
    if 'Item' in df.columns:
        item_df = pd.DataFrame(df['Item'].fillna(0).astype(float), columns=['Item'])
        logging.info("Coluna 'Item' convertida para numérico")
    else:
        logging.warning("Coluna 'Item' não encontrada")
        item_df = pd.DataFrame(index=range(len(df)))
except Exception as e:
    logging.error(f"Erro ao converter 'Item' para numérico: {e}")
    item_df = pd.DataFrame(index=range(len(df)))

# Combinar todas as features em um único DataFrame
X = pd.concat([desc_df, cat_df, item_df], axis=1)
logging.info(f"DataFrame de features combinado. Dimensões: {X.shape}")

# Preparar os alvos (substituir vazios por NaN)
y = df[targets].replace('', np.nan)

# Identificar linhas com pelo menos um valor não-nulo nos targets (para treino)
# Alterado para identificar linhas com pelo menos um valor não nulo nos targets
has_target_values = ~y.isna().all(axis=1)
complete_rows = y[has_target_values].index
incomplete_rows = y[~has_target_values].index

logging.info(f"Linhas com valores para treino: {len(complete_rows)}")
logging.info(f"Linhas para previsão: {len(incomplete_rows)}")

# Verificar se há dados suficientes para treino
min_train_samples = 10  # Definir um limiar mínimo para treinamento
if len(complete_rows) < min_train_samples:
    logging.warning(f"Dados insuficientes para treinamento: {len(complete_rows)} < {min_train_samples}")
    print(f"ATENÇÃO: Apenas {len(complete_rows)} amostras para treinamento. Os resultados podem não ser confiáveis.")

# Separar dados de treino e teste
X_train = X.loc[complete_rows]
y_train = y.loc[complete_rows].fillna({'FatorK1': 1.0, 'FatorK2': 1.0, 'FatorK3': 1.0, 
                                      'FatorK4': 1.0, 'FatorK5': 1.0, 'FatorK6': 1.0})
X_pred = X.loc[incomplete_rows] if len(incomplete_rows) > 0 else pd.DataFrame()

2025-04-11 13:33:24,895 - INFO - Valores nulos na coluna 'Descrição Linha de Serviço' preenchidos com ''
2025-04-11 13:33:24,899 - INFO - Valores nulos na coluna 'Classificação' preenchidos com ''
2025-04-11 13:33:24,901 - INFO - Valores nulos na coluna 'Fornecedor' preenchidos com ''
2025-04-11 13:33:24,903 - INFO - Valores nulos na coluna 'Moeda' preenchidos com ''
2025-04-11 13:33:24,904 - INFO - Valores nulos na coluna 'Item' preenchidos com 0
2025-04-11 13:33:24,999 - INFO - TF-IDF aplicado com sucesso. Dimensões: (2571, 50)
2025-04-11 13:33:25,014 - INFO - Codificação One-Hot aplicada com sucesso. Dimensões: (2571, 41)
2025-04-11 13:33:25,016 - INFO - Coluna 'Item' convertida para numérico
2025-04-11 13:33:25,020 - INFO - DataFrame de features combinado. Dimensões: (2571, 92)
2025-04-11 13:33:25,027 - INFO - Linhas com valores para treino: 1221
2025-04-11 13:33:25,028 - INFO - Linhas para previsão: 1350



Quantidade de valores nulos por coluna:
Classificação                    0
Sub-Classificação                6
Projeto                          6
Contrato Juridico                2
Contrato SAP                     0
Item                             0
Linha                            0
Descrição Linha de Serviço       0
Unidade de Medida                0
Qtde Prevista                    0
Qtde Realizada                   0
Preço Base                       0
Valor Líquido Realizado          0
Valor PIS                        0
Valor COFINS                     0
Valor ICMS                       0
Valor Bruto Realizado            0
Fornecedor                       0
Data Base                        0
Moeda                            0
idLinhaServico                   0
Indice1                       1350
Peso1                         1350
FatorK1                       1350
Indice2                       1413
Peso2                         1413
FatorK2                       1413
Indice3       

# **Bloco 3: Treinamento do Modelo de Machine Learning**


In [9]:
# Separar índices e pesos para modelagem separada
indices_cols = [col for col in targets if 'Indice' in col]
pesos_cols = [col for col in targets if 'Peso' in col]
fatork_cols = [col for col in targets if 'FatorK' in col]

# Função para verificar a qualidade do modelo usando validação cruzada
def evaluate_model(model, X, y, model_type="classificação"):
    if len(X) >= 10:  # Verificar se há amostras suficientes para CV
        try:
            scores = cross_val_score(model, X, y, cv=min(5, len(X)))
            logging.info(f"Validação cruzada para {model_type}: {scores.mean():.4f} (±{scores.std():.4f})")
            return scores.mean()
        except Exception as e:
            logging.warning(f"Erro na validação cruzada do modelo de {model_type}: {e}")
            return None
    else:
        logging.warning(f"Amostras insuficientes para validação cruzada do modelo de {model_type}")
        return None

# Treinar modelos apenas se houver dados suficientes
if len(complete_rows) >= min_train_samples:
    try:
        # Modelo para prever índices (classificação)
        # Filtrar apenas as linhas que têm pelo menos um índice não-nulo
        indices_data = y_train[indices_cols].fillna('Nenhum')
        has_indices = (indices_data != 'Nenhum').any(axis=1)
        
        if has_indices.any():
            clf = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
            clf.fit(X_train[has_indices], indices_data[has_indices])
            evaluate_model(clf, X_train[has_indices], indices_data[has_indices], "índices")
            logging.info("Modelo de classificação para índices treinado com sucesso")
        else:
            clf = None
            logging.warning("Nenhum dado válido para treinar o modelo de índices")
        
        # Modelo para prever pesos (regressão)
        # Filtrar apenas as linhas que têm pelo menos um peso não-nulo
        pesos_data = y_train[pesos_cols].fillna(0)
        has_pesos = (pesos_data > 0).any(axis=1)
        
        if has_pesos.any():
            reg = RandomForestRegressor(n_estimators=100, random_state=42)
            reg.fit(X_train[has_pesos], pesos_data[has_pesos])
            evaluate_model(reg, X_train[has_pesos], pesos_data[has_pesos], "pesos")
            logging.info("Modelo de regressão para pesos treinado com sucesso")
        else:
            reg = None
            logging.warning("Nenhum dado válido para treinar o modelo de pesos")
            
    except Exception as e:
        logging.error(f"Erro no treinamento dos modelos: {e}")
        clf, reg = None, None
else:
    logging.warning("Dados insuficientes para treinar modelos")
    clf, reg = None, None

# Prever índices e pesos para linhas incompletas (se houver modelos treinados e dados para prever)
if len(incomplete_rows) > 0 and X_pred.shape[0] > 0:
    # Inicializar DataFrame de previsões com valores vazios
    pred_df = pd.DataFrame(index=incomplete_rows, columns=targets)
    
    # Prever índices se o modelo foi treinado
    if clf is not None:
        try:
            indices_pred = clf.predict(X_pred)
            for i, col in enumerate(indices_cols):
                pred_df[col] = indices_pred[:, i]
            logging.info("Previsão de índices concluída")
        except Exception as e:
            logging.error(f"Erro na previsão de índices: {e}")
            # Usar 'Nenhum' como fallback para índices
            for col in indices_cols:
                pred_df[col] = 'Nenhum'
    else:
        # Usar 'Nenhum' como fallback para índices se não há modelo
        for col in indices_cols:
            pred_df[col] = 'Nenhum'
    
    # Prever pesos se o modelo foi treinado
    if reg is not None:
        try:
            pesos_pred = reg.predict(X_pred)
            
            # Normalizar pesos para soma <=
            def normalize_weights(weights):
                # Substituir NaN com 0 para evitar problemas
                weights = np.nan_to_num(weights)
                total = np.sum(weights)
                if total > 1:
                    # Normalizar apenas valores não-nulos
                    return weights / total if total > 0 else weights
                return weights
            
            pesos_pred_normalized = np.apply_along_axis(normalize_weights, 1, pesos_pred)
            
            for i, col in enumerate(pesos_cols):
                pred_df[col] = pesos_pred_normalized[:, i]
            logging.info("Previsão de pesos concluída e normalizada")
        except Exception as e:
            logging.error(f"Erro na previsão de pesos: {e}")
            # Usar 0 como fallback para pesos
            for col in pesos_cols:
                pred_df[col] = 0
    else:
        # Usar 0 como fallback para pesos se não há modelo
        for col in pesos_cols:
            pred_df[col] = 0
            
    # Garantir consistência entre índices e pesos
    # Se um peso for > 0, mas não há índice, atribuir um índice padrão
    # Se um índice for 'Nenhum' ou vazio, zerar o peso correspondente
    for i in range(1, 7):  # Para cada par índice/peso (1 a 6)
        indice_col = f'Indice{i}'
        peso_col = f'Peso{i}'
        
        if indice_col in pred_df.columns and peso_col in pred_df.columns:
            # Caso 1: Peso > 0 mas sem índice - atribuir índice padrão "IGP"
            mask = (pred_df[peso_col] > 0) & ((pred_df[indice_col] == '') | (pred_df[indice_col] == 'Nenhum'))
            if mask.any():
                pred_df.loc[mask, indice_col] = "IGP"  # Índice padrão
                logging.info(f"Atribuído índice padrão 'IGP' para {mask.sum()} linhas com {peso_col} > 0 sem índice")
            
            # Caso 2: Índice vazio ou 'Nenhum', zerar o peso correspondente
            mask = ((pred_df[indice_col] == '') | (pred_df[indice_col] == 'Nenhum'))
            if mask.any():
                pred_df.loc[mask, peso_col] = 0
                logging.info(f"Zerados {mask.sum()} pesos onde {indice_col} está vazio ou é 'Nenhum'")
    
    # Verificar novamente se a soma dos pesos é <= 1 após as correções
    for idx in pred_df.index:
        peso_values = [pred_df.loc[idx, f'Peso{i}'] for i in range(1, 7)]
        peso_values = [p for p in peso_values if not pd.isna(p)]
        total = sum(peso_values)
        
        if total > 1:
            for i in range(1, 7):
                peso_col = f'Peso{i}'
                if peso_col in pred_df.columns and not pd.isna(pred_df.loc[idx, peso_col]) and pred_df.loc[idx, peso_col] > 0:
                    pred_df.loc[idx, peso_col] = pred_df.loc[idx, peso_col] / total
            
            logging.info(f"Pesos renormalizados para linha {idx} após correções de consistência")
    
    # Definir FatorK como 1.0 para todas as linhas
    for col in fatork_cols:
        pred_df[col] = 1.0
        
    logging.info("DataFrame de previsões criado com sucesso")
else:
    pred_df = pd.DataFrame()
    logging.info("Nenhuma linha para previsão ou modelos não treinados")

2025-04-11 13:34:15,101 - INFO - Validação cruzada para índices: 0.4056 (±0.2312)
2025-04-11 13:34:15,102 - INFO - Modelo de classificação para índices treinado com sucesso
2025-04-11 13:34:17,237 - INFO - Validação cruzada para pesos: -3.5357 (±4.6310)
2025-04-11 13:34:17,239 - INFO - Modelo de regressão para pesos treinado com sucesso
2025-04-11 13:34:17,314 - INFO - Previsão de índices concluída
2025-04-11 13:34:17,351 - INFO - Previsão de pesos concluída e normalizada
2025-04-11 13:34:17,359 - INFO - Atribuído índice padrão 'IGP' para 108 linhas com Peso2 > 0 sem índice
2025-04-11 13:34:17,362 - INFO - Zerados 336 pesos onde Indice2 está vazio ou é 'Nenhum'
2025-04-11 13:34:17,366 - INFO - Atribuído índice padrão 'IGP' para 296 linhas com Peso3 > 0 sem índice
2025-04-11 13:34:17,368 - INFO - Zerados 916 pesos onde Indice3 está vazio ou é 'Nenhum'
2025-04-11 13:34:17,371 - INFO - Atribuído índice padrão 'IGP' para 96 linhas com Peso4 > 0 sem índice
2025-04-11 13:34:17,379 - INFO - Z

# **Bloco 4: Combinar Previsões com Dados Originais**

In [10]:
# Copiar o DataFrame original
df_filled = df.copy()

# Flag para verificar se houve previsões aplicadas
predictions_applied = False

# Registrar quais células foram preenchidas (para colorir no Excel depois)
filled_cells = {}  # Dicionário para armazenar posições (linha, coluna) das células preenchidas

# Atualizar apenas as linhas incompletas com as previsões (se houver)
if not pred_df.empty:
    for idx in pred_df.index:
        # Primeiro verificar se precisamos aplicar qualquer previsão nesta linha
        any_missing = False
        for col in targets:
            is_empty = pd.isna(df_filled.loc[idx, col]) or df_filled.loc[idx, col] == ''
            if is_empty:
                any_missing = True
                break
        
        if any_missing:
            # Primeiro preenchimento: encontrar índices válidos e seus pesos correspondentes
            valid_indices_and_weights = []
            
            # Verificar valores existentes (não nulos e não vazios) nos dados originais
            for i in range(1, 7):
                indice_col = f'Indice{i}'
                peso_col = f'Peso{i}'
                indice_value = df_filled.loc[idx, indice_col]
                peso_value = df_filled.loc[idx, peso_col]
                
                indice_filled = not (pd.isna(indice_value) or indice_value == '' or indice_value == 'Nenhum')
                peso_filled = not (pd.isna(peso_value) or peso_value == 0)
                
                # Adicionar à lista se já estiver preenchido
                if indice_filled or peso_filled:
                    valid_indices_and_weights.append((indice_col, peso_col))
            
            # Agora, preencher os valores ausentes
            for i in range(1, 7):
                indice_col = f'Indice{i}'
                peso_col = f'Peso{i}'
                fatork_col = f'FatorK{i}'
                
                # Verificar se o par de índice e peso é válido para atualizar
                if (indice_col, peso_col) not in valid_indices_and_weights:
                    # Verificar se existem valores nas previsões
                    indice_pred = pred_df.loc[idx, indice_col] if indice_col in pred_df.columns else None
                    peso_pred = pred_df.loc[idx, peso_col] if peso_col in pred_df.columns else None
                    
                    # Só preencher se tiver previsão com valor não nulo/vazio
                    valid_indice = indice_pred is not None and indice_pred != '' and indice_pred != 'Nenhum'
                    valid_peso = peso_pred is not None and peso_pred > 0
                    
                    # Aplicar previsões apenas para pares válidos (índice e peso)
                    if valid_indice and valid_peso:
                        # Preencher índice
                        is_empty_indice = pd.isna(df_filled.loc[idx, indice_col]) or df_filled.loc[idx, indice_col] == ''
                        if is_empty_indice:
                            if idx not in filled_cells:
                                filled_cells[idx] = []
                            filled_cells[idx].append(indice_col)
                            df_filled.loc[idx, indice_col] = indice_pred
                            predictions_applied = True
                        
                        # Preencher peso
                        is_empty_peso = pd.isna(df_filled.loc[idx, peso_col]) or df_filled.loc[idx, peso_col] == 0
                        if is_empty_peso:
                            if idx not in filled_cells:
                                filled_cells[idx] = []
                            filled_cells[idx].append(peso_col)
                            df_filled.loc[idx, peso_col] = peso_pred
                            predictions_applied = True
                        
                        # Preencher fatorK se necessário
                        is_empty_fatork = pd.isna(df_filled.loc[idx, fatork_col])
                        if is_empty_fatork:
                            if idx not in filled_cells:
                                filled_cells[idx] = []
                            filled_cells[idx].append(fatork_col)
                            df_filled.loc[idx, fatork_col] = 1.0
                            predictions_applied = True
    
    logging.info("Previsões combinadas com dados originais")
else:
    logging.info("Nenhuma previsão para combinar com os dados originais")

# Garantir que os valores sejam consistentes (ex.: substituir 'Nenhum' por vazio nos índices)
for col in indices_cols:
    if col in df_filled.columns:
        mask = df_filled[col] == 'Nenhum'
        if mask.any():
            df_filled.loc[mask, col] = ''
            logging.info(f"Valores 'Nenhum' na coluna {col} substituídos por vazio")

2025-04-11 13:34:43,156 - INFO - Previsões combinadas com dados originais


# **Bloco 5: Exportar o Arquivo com Cores Diferenciadas**

In [11]:
# Definir nome do arquivo de saída
output_file = 'output_preenchido_corrigido.xlsx'

try:
    # Criar um workbook e worksheet no openpyxl
    wb = Workbook()
    ws = wb.active
    
    # Definir cores para cada grupo
    fills = {
        'Indice1': PatternFill(start_color='ADD8E6', end_color='ADD8E6', fill_type='solid'),  # Azul claro
        'Peso1': PatternFill(start_color='ADD8E6', end_color='ADD8E6', fill_type='solid'),
        'FatorK1': PatternFill(start_color='ADD8E6', end_color='ADD8E6', fill_type='solid'),
        'Indice2': PatternFill(start_color='90EE90', end_color='90EE90', fill_type='solid'),  # Verde claro
        'Peso2': PatternFill(start_color='90EE90', end_color='90EE90', fill_type='solid'),
        'FatorK2': PatternFill(start_color='90EE90', end_color='90EE90', fill_type='solid'),
        'Indice3': PatternFill(start_color='FFFFE0', end_color='FFFFE0', fill_type='solid'),  # Amarelo claro
        'Peso3': PatternFill(start_color='FFFFE0', end_color='FFFFE0', fill_type='solid'),
        'FatorK3': PatternFill(start_color='FFFFE0', end_color='FFFFE0', fill_type='solid'),
        'Indice4': PatternFill(start_color='FFA07A', end_color='FFA07A', fill_type='solid'),  # Laranja claro
        'Peso4': PatternFill(start_color='FFA07A', end_color='FFA07A', fill_type='solid'),
        'FatorK4': PatternFill(start_color='FFA07A', end_color='FFA07A', fill_type='solid'),
        'Indice5': PatternFill(start_color='DDA0DD', end_color='DDA0DD', fill_type='solid'),  # Roxo claro
        'Peso5': PatternFill(start_color='DDA0DD', end_color='DDA0DD', fill_type='solid'),
        'FatorK5': PatternFill(start_color='DDA0DD', end_color='DDA0DD', fill_type='solid'),
        'Indice6': PatternFill(start_color='F0E68C', end_color='F0E68C', fill_type='solid'),  # Amarelo ouro
        'Peso6': PatternFill(start_color='F0E68C', end_color='F0E68C', fill_type='solid'),
        'FatorK6': PatternFill(start_color='F0E68C', end_color='F0E68C', fill_type='solid'),
    }
    
    # Escrever cabeçalhos
    for col_num, value in enumerate(df_filled.columns, 1):
        ws.cell(row=1, column=col_num, value=value)
    
    # Escrever dados e aplicar cores para valores preenchidos pelo modelo
    for row_num, (idx, row) in enumerate(df_filled.iterrows(), 2):
        for col_num, (col_name, value) in enumerate(row.items(), 1):
            cell = ws.cell(row=row_num, column=col_num, value=value)
            
            # Verificar se essa célula foi preenchida pelo modelo
            if idx in filled_cells and col_name in filled_cells[idx] and col_name in fills:
                cell.fill = fills[col_name]
    
    # Salvar o arquivo
    wb.save(output_file)
    logging.info(f"Arquivo '{output_file}' gerado com sucesso!")
    print(f"Arquivo '{output_file}' gerado com sucesso!")
    
    # Exibir estatísticas
    if predictions_applied:
        n_cells = sum(len(cols) for cols in filled_cells.values())
        n_rows = len(filled_cells)
        print(f"Total de {n_cells} células preenchidas em {n_rows} linhas.")
    else:
        print("Nenhuma célula foi preenchida pelo modelo.")
        
except Exception as e:
    logging.error(f"Erro ao criar arquivo Excel: {e}")
    print(f"Erro ao gerar o arquivo Excel: {e}")

2025-04-11 13:34:52,015 - INFO - Arquivo 'output_preenchido_corrigido.xlsx' gerado com sucesso!


Arquivo 'output_preenchido_corrigido.xlsx' gerado com sucesso!
Total de 9189 células preenchidas em 1350 linhas.
